In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pred
import numpy as np
from CONSTANT import *
from tools import *
from torch.utils.data import (
    TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

In [29]:
batch_size = 64
learning_rate = 1e-2
num_epochs = 10 # 训练次数
# 判断GPU是否可用
use_gpu = torch.cuda.is_available()

# # 下载训练集 MNIST 手写数字训练集
# # 数据是datasets类型的
# train_dataset = datasets.FashionMNIST(
#     root='./datasets', train=True, transform=transforms.ToTensor(), download=True)

# test_dataset = datasets.FashionMNIST(
#     root='./datasets', train=False, transform=transforms.ToTensor())
# #　将数据处理成 DataLoader
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # 选择打乱数据
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # 选择不打乱数据

In [13]:
class DatasetProvider(torch.utils.data.Dataset):
    def __init__(self, target, file_name, train_index, test_index, batch_size):
        data = pd.read_csv(file_name)
        X, y = join_signals(data, target=target)

        xtrain, ytrain, xtest, ytest = X[train_index], y[train_index], X[test_index], y[test_index]

        self.xtrain = torch.tensor(xtrain, dtype=torch.float32)
        self.xtest = torch.tensor(xtest, dtype=torch.float32)

        self.ytrain = torch.tensor(ytrain, dtype=torch.long)
        self.ytest = torch.tensor(ytest, dtype=torch.long)

        print(self.xtrain.shape, self.ytrain.shape, self.xtest.shape, self.ytest.shape)

    def get_data(self):
        train_set = TensorDataset(self.xtrain, self.ytrain)
        test_set = TensorDataset(self.xtest, self.ytest)

        train_loader = DataLoader(train_set,batch_size=batch_size, shuffle=True, drop_last=False)
        test_loader = DataLoader(test_set, batch_size=batch_size,drop_last=False)

        return train_loader, test_loader

In [5]:
spliter = load_model(
        r'./processed_signal/HKU956/400_4s_step_2s_spliter.pkl')

In [6]:
for k in spliter['loso']:
    train_index = k['train_index']
    test_index = k['test_index']
    break

In [34]:
dataprovider = DatasetProvider(target='valence_label',
                               file_name=r'./processed_signal/HKU956/400_4s_step_2s.csv',
                               train_index=train_index,
                               test_index=test_index,
                               batch_size=16
                            )

torch.Size([18089, 4, 400]) torch.Size([18089, 1]) torch.Size([4638, 4, 400]) torch.Size([4638, 1])


In [35]:
train_loader, test_loader = dataprovider.get_data()

In [42]:
import torch.nn.functional as F

In [52]:
class neuralNetwork(nn.Module):
    def __init__(self, in_dim, layer_num, n_hidden_1, n_hidden_2, out_dim):
        super(neuralNetwork, self).__init__() # super() 函数是用于调用父类(超类)的一个方法
# Sequential()表示将一个有序的模块写在一起，也就相当于将神经网络的层按顺序放在一起，这样可以方便结构显示
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True)) # 表示使用ReLU激活函数
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True))
        self.layer_m = nn.ModuleList([nn.Linear(n_hidden_2, n_hidden_2) for _ in range(layer_num)])
        self.layer3 = nn.Sequential(
            nn.Linear(n_hidden_2, out_dim),
            nn.ReLU(True))

# 定义向前传播
    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = self.layer1(x)
        x = self.layer2(x)
        for m in self.layer_m:
            x = m(x)
            x = F.relu(x)
        x = self.layer3(x)
        return x

In [58]:
model = neuralNetwork(4*400, 5, 1024, 512, 2)
if use_gpu:
    model = model.cuda()

for param in model.parameters():
        nn.init.normal_(param, mean=0, std=0.01)

In [49]:
# xtest = dataprovider.xtest[:100]
# xtest.shape

torch.Size([100, 4, 400])

In [64]:
criterion = nn.CrossEntropyLoss() # 定义损失函数类型，使用交叉熵
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # 定义优化器，使用随机梯度下降

In [65]:
# 开始模型训练
for epoch in range(num_epochs):
    # print('*' * 10)
    # print(f'epoch {epoch+1}')
    running_loss = 0.0 # 初始值
    running_acc = 0.0
    for i, batch in enumerate(train_loader, 1): # 枚举函数enumerate返回下标和值
        data, label = batch

        label = label.reshape(1, -1)[0]
        # 使用GPU？
        if use_gpu:
            data = data.cuda()
            label = label.cuda()
        # 向前传播
        out = model(data) # 前向传播
        loss = criterion(out, label) # 计算loss
        running_loss += loss.item() # loss求和
        _, pred = torch.max(out, 1)
        running_acc += (pred == label).float().mean()
        # 向后传播
        optimizer.zero_grad() # 梯度归零
        loss.backward() # 后向传播
        optimizer.step() # 更新参数

        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    # for name, parms in model.named_parameters():
    #     if name in ['layer3.0.bias', 'layer3.0.bias']:
    #         print(name, 'grad_requirs:', parms.requires_grad, 'weight', torch.mean(parms.data),'grad_value:', torch.mean(parms.grad))

Finish 1 epoch, Loss: 0.690465, Acc: 0.537066
Finish 2 epoch, Loss: 0.690460, Acc: 0.537190
Finish 3 epoch, Loss: 0.690496, Acc: 0.537035
Finish 4 epoch, Loss: 0.690480, Acc: 0.537035
Finish 5 epoch, Loss: 0.690418, Acc: 0.537283
Finish 6 epoch, Loss: 0.690510, Acc: 0.536973
Finish 7 epoch, Loss: 0.690456, Acc: 0.537035
Finish 8 epoch, Loss: 0.690479, Acc: 0.537066
Finish 9 epoch, Loss: 0.690480, Acc: 0.536973
Finish 10 epoch, Loss: 0.690464, Acc: 0.537159


In [41]:
for i, data in enumerate(train_loader, 1): # 枚举函数enumerate返回下标和值
        img, label = data
        print(img.requires_grad)
        print(img.grad)
        break

False
None


In [17]:
img.shape

torch.Size([64, 1, 28, 28])